In [1]:
# 1. Importamos todas las librerías que creemos útiles

import io # Provee diferentes tipos de E/S: texto E/S, binario E/S e E/S.

import os # Leer o escribir un archivo.

import math # Proporciona acceso a las funciones matemáticas definidas en C.

import csv # Para cargar y manejar CSV.

import xlrd # Funcionalidades de excel en Python (1/2).

import xlwt # Funcionalidades de excel en Python (2/2).

import sklearn # Funcionalidades de Machine Learning.

import numpy as np # Manejo de datos extremadamente rápido cálculo numérico.

import scipy as sp # Módulos para optimización, álgebra lineal, etc.

import pandas as pd # Módulos de gestión de ficheros y dataframes.

import matplotlib.pyplot as plt # Generación de gráficos.

import statsmodels.api as sm # Módulo estadístico.

import random # Para fijar resultados.

In [2]:
# 2.1 Leo el fichero5.

fichero5 = pd.read_csv(r'C:\Users\jlc15\Desktop\MECOFIN\2º Cuatrimestre\TFM\Y - Ficheros Python\fichero5.csv')

fichero5

,Gender,Education,Work Exp,Programming Exp,Assurance,Matching,Status
0,0,1,0,0,0.50,0,0
1,1,1,0,1,0.67,1,1
2,1,1,1,0,0.83,1,1
3,1,1,0,0,0.50,0,0
4,1,0,1,0,0.33,0,1
...,...,...,...,...,...,...,...
824,1,1,0,0,0.50,1,1
825,1,1,1,0,0.83,1,1
826,1,1,0,1,0.67,0,0
827,1,1,0,0,0.50,1,1


In [3]:
# 2.2 Eliminamos la columna Assurance.

# Esto lo hacemos porque ya hemos estudiado todas las variables
# y lo que cada una supone en el modelo. En este caso Assurance
# a nivel estadístico es muy interesante porque nos muestra
# la frecuencia y distribución de los perfiles.

# Sin embargo, Assurance no deja de ser una columna que explica
# Matching de forma directa, por lo que prescindiremos de ella
# para evitar redundancias en el modelo.

fichero5 = fichero5.drop(['Assurance'], axis=1)

fichero5

,Gender,Education,Work Exp,Programming Exp,Matching,Status
0,0,1,0,0,0,0
1,1,1,0,1,1,1
2,1,1,1,0,1,1
3,1,1,0,0,0,0
4,1,0,1,0,0,1
...,...,...,...,...,...,...
824,1,1,0,0,1,1
825,1,1,1,0,1,1
826,1,1,0,1,0,0
827,1,1,0,0,1,1


In [4]:
# 2.3 Exportamos el dataframe a otro fichero formato CSV diferente.

fichero5.to_csv('fichero6.csv', index=False)

In [5]:
# 2.4 Cargamos y leemos el nuevo fichero6.

fichero6 = pd.read_csv(r'C:\Users\jlc15\Desktop\MECOFIN\2º Cuatrimestre\TFM\Y - Ficheros Python\fichero6.csv')

fichero6

,Gender,Education,Work Exp,Programming Exp,Matching,Status
0,0,1,0,0,0,0
1,1,1,0,1,1,1
2,1,1,1,0,1,1
3,1,1,0,0,0,0
4,1,0,1,0,0,1
...,...,...,...,...,...,...
824,1,1,0,0,1,1
825,1,1,1,0,1,1
826,1,1,0,1,0,0
827,1,1,0,0,1,1


In [9]:
# 3.1 Analizamos antes de aplicar el Logit si el modelo posee o no
#     heterocedasticidad.

# Para ello, aplicamos el test de White.

from statsmodels.stats.diagnostic import het_white

y = fichero6['Status']
x = fichero6[['Gender', 'Education', 'Work Exp', 'Programming Exp']]

x = sm.add_constant(x)

model = sm.Logit(y, x).fit()

white_test = het_white(model.resid_response, model.model.exog)

labels = ['Estadístico de prueba', 'p-valor del estadístico de prueba', 'Estadístico F', 'p-valor del estadístico F']

print(dict(zip(labels, white_test)))

Optimization terminated successfully.
         Current function value: 0.612652
         Iterations 5
{'Estadístico de prueba': 16.419665850136397, 'p-valor del estadístico de prueba': 0.0882327410041312, 'Estadístico F': 1.652918007111709, 'p-valor del estadístico F': 0.08760357303657892}


In [ ]:
# 3.2 Conclusiones:

# a) Según el p-value (0.0882), al no ser el resultado inferior al 5% establecido,
#    no se rechaza la hipotesis nula (homocedasticidad), por lo que no se 
#    puede afirmar que exista heterocedasticidad.

# b) Además, sólo se han necesitado 5 iteraciones, luego esto es un signo de
#    que el algoritmo es eficiente.

# El resto de resultados no se estudian porque no son relevantes en nuestro
# estudio.

In [23]:
# 4.1 Implementamos el modelo econométrico Logit.

# Sólo vamos a incluir las variables originales que explican el modelo,
# ya que la correlación entre la variable endógena (Status) y Matching
# es muy alta. Es decir, excluimos Matching para estudiar las variables
# originales en el modelo sin la influencia de Matching.

# Tampoco tiene sentido incluir Matching en el análisis econométrico
# porque está formada indirectamente por las variables originales vía
# columna Assurance.

# Luego, el objetivo es explicar las variables originales para dar
# sentido a la creación de Matching.

# Parte A: hacemos directamente la regresión logística.

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

x = fichero6[['Gender', 'Education', 'Work Exp', 'Programming Exp']]
y = fichero6['Status']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

log_reg = LogisticRegression()

log_reg.fit(x_train, y_train)

score = log_reg.score(x_test, y_test)

score

0.7355769230769231

In [7]:
# 4.2 Conclusiones (A):

# a) Este método se calcula como el número de predicciones correctas dividido por el número 
#    total de predicciones, el cual ha dado un resultado de 0.73. Lo que significa que el modelo  
#    logit hizo predicciones correctas en el 73% de los casos en los datos de prueba. Esto 
#    puede considerarse un rendimiento aceptable.

# b) Por otra parte, en logit sabemos su signo, que en este caso es positivo. Esto significa
#    que el modelo contra más cerca esté de 1 mejor (en términos de significatividad).

# c) Por último cabe destacar que este tipo de análisis no son posibles con conjunto de datos
#    reducidos, por lo que cobra más sentido la ampliación realizada del dataset en el capítulo 1.

In [8]:
# 4.3 Aplicamos la T-Student con Logit y analizamos los P-Value.

# Parte B: Regresión logística en Python para poner a prueba el 
#          modelo y obtener los p-values de las variables utilizando 
#          una t-student.

import statsmodels.api as sm

v_endogena = fichero6['Status']

v_exogenas = sm.add_constant(fichero6[['Gender', 'Education', 'Work Exp', 'Programming Exp']])

modelo = sm.Logit(v_endogena, v_exogenas)

resultados = modelo.fit()

print(resultados.summary())

Optimization terminated successfully.
         Current function value: 0.612652
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                 Status   No. Observations:                  829
Model:                          Logit   Df Residuals:                      824
Method:                           MLE   Df Model:                            4
Date:                Sat, 15 Apr 2023   Pseudo R-squ.:                 0.01349
Time:                        23:44:46   Log-Likelihood:                -507.89
converged:                       True   LL-Null:                       -514.83
Covariance Type:            nonrobust   LLR p-value:                  0.007668
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.2261      0.245      0.924      0.356      -0.254       0.706
Gender        

In [9]:
# 4.4 Conclusiones (B):

# a) Con estos resultados afirmamos que Education es la única variable
#    que se puede considerar significativa y que el resto teóricamente
#    no lo son.

# b) Sin embargo, el LLR (Likelihood Ratio Test) mide la significancia 
# estadística del modelo Logit. Un valor p bajo indica que el modelo es 
# estadísticamente significativo. En nuestro caso es muy bajo (0.0076).

# b) También reafirmamos la forma en distribuir los pesos para el Score
#    hacia Work Exp y Programming Exp.

# c) Por último destacamos que la variable menos significativa es el género.